In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print("TensorFlow version:", tf.__version__)
print("Libraries imported successfully!")


In [ ]:
# Simple RNN Implementation from Scratch
class SimpleRNN:
    def __init__(self, input_size, hidden_size, output_size):
        """
        Initialize a simple RNN
        
        Parameters:
        input_size: dimension of input features
        hidden_size: dimension of hidden state
        output_size: dimension of output
        """
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        # Initialize weights with small random values
        self.W_xh = np.random.randn(input_size, hidden_size) * 0.01
        self.W_hh = np.random.randn(hidden_size, hidden_size) * 0.01
        self.W_hy = np.random.randn(hidden_size, output_size) * 0.01
        
        # Initialize biases
        self.b_h = np.zeros((1, hidden_size))
        self.b_y = np.zeros((1, output_size))
        
        # Initialize hidden state
        self.h = np.zeros((1, hidden_size))
    
    def forward(self, x):
        """
        Forward pass through the RNN
        
        Parameters:
        x: input sequence (sequence_length, input_size)
        
        Returns:
        outputs: output sequence (sequence_length, output_size)
        hidden_states: hidden states (sequence_length, hidden_size)
        """
        sequence_length = x.shape[0]
        
        outputs = []
        hidden_states = []
        
        # Process each time step
        for t in range(sequence_length):
            # Update hidden state
            self.h = np.tanh(np.dot(x[t:t+1], self.W_xh) + 
                           np.dot(self.h, self.W_hh) + self.b_h)
            
            # Compute output
            output = np.dot(self.h, self.W_hy) + self.b_y
            
            outputs.append(output)
            hidden_states.append(self.h.copy())
        
        return np.array(outputs), np.array(hidden_states)
    
    def reset_hidden_state(self):
        """Reset hidden state to zeros"""
        self.h = np.zeros((1, self.hidden_size))

# Test the simple RNN
print("Testing Simple RNN Implementation:")
print("-" * 40)

# Create a simple RNN
rnn = SimpleRNN(input_size=3, hidden_size=5, output_size=2)

# Create dummy input sequence
sequence_length = 4
input_sequence = np.random.randn(sequence_length, 3)

print(f"Input sequence shape: {input_sequence.shape}")

# Forward pass
outputs, hidden_states = rnn.forward(input_sequence)

print(f"Output sequence shape: {outputs.shape}")
print(f"Hidden states shape: {hidden_states.shape}")

print("\nInput at each time step:")
for t in range(sequence_length):
    print(f"Time {t}: {input_sequence[t]}")

print("\nHidden states at each time step:")
for t in range(sequence_length):
    print(f"Time {t}: {hidden_states[t].flatten()}")

print("\nOutputs at each time step:")
for t in range(sequence_length):
    print(f"Time {t}: {outputs[t].flatten()}")


In [ ]:
# Visualizing RNN Unfolding and Hidden State Evolution
def visualize_rnn_unfolding(sequence, hidden_states):
    """
    Visualize how hidden states evolve through time
    """
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
    
    # Plot input sequence
    ax1.plot(sequence, marker='o', linewidth=2, markersize=8)
    ax1.set_title('Input Sequence Over Time', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Time Step')
    ax1.set_ylabel('Input Value')
    ax1.grid(True, alpha=0.3)
    
    # Plot hidden state evolution
    hidden_states_2d = hidden_states.squeeze()
    for i in range(hidden_states_2d.shape[1]):
        ax2.plot(hidden_states_2d[:, i], marker='o', linewidth=2, 
                markersize=6, label=f'Hidden Unit {i+1}')
    
    ax2.set_title('Hidden State Evolution Through Time', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Time Step')
    ax2.set_ylabel('Hidden State Value')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Create a simple time series for demonstration
time_steps = np.arange(10)
# Create a sine wave with some noise
input_sequence = np.sin(time_steps * 0.5) + 0.1 * np.random.randn(10)

# Reshape for RNN (each time step has 1 feature)
input_sequence_reshaped = input_sequence.reshape(-1, 1)

# Create and test RNN
rnn_demo = SimpleRNN(input_size=1, hidden_size=3, output_size=1)
outputs, hidden_states = rnn_demo.forward(input_sequence_reshaped)

print("Demonstrating RNN Unfolding:")
print(f"Input sequence: {input_sequence}")
print(f"Hidden states shape: {hidden_states.shape}")

# Visualize the unfolding
visualize_rnn_unfolding(input_sequence, hidden_states)


In [ ]:
# Building RNN with TensorFlow/Keras
def create_simple_rnn_model(input_shape, hidden_units=50, output_units=1):
    """
    Create a simple RNN model using Keras
    
    Parameters:
    input_shape: (sequence_length, features)
    hidden_units: number of RNN units
    output_units: number of output units
    
    Returns:
    model: compiled Keras model
    """
    model = keras.Sequential([
        keras.layers.SimpleRNN(
            hidden_units, 
            input_shape=input_shape,
            return_sequences=False,  # Return only the last output
            activation='tanh'
        ),
        keras.layers.Dense(output_units, activation='linear')
    ])
    
    model.compile(
        optimizer='adam',
        loss='mse',
        metrics=['mae']
    )
    
    return model

# Create sample sequential data for demonstration
def generate_sine_sequence(n_samples, sequence_length, n_features):
    """Generate sine wave sequences for testing"""
    X = []
    y = []
    
    for i in range(n_samples):
        # Generate sine wave with random phase and frequency
        phase = np.random.random() * 2 * np.pi
        freq = 0.1 + np.random.random() * 0.4
        
        # Create sequence
        t = np.linspace(0, sequence_length, sequence_length)
        sequence = np.sin(freq * t + phase)
        
        # Add noise
        sequence += 0.1 * np.random.randn(sequence_length)
        
        # Prepare input (all but last) and target (last value)
        X.append(sequence[:-1].reshape(-1, 1))
        y.append(sequence[-1])
    
    return np.array(X), np.array(y)

# Generate training data
print("Generating training data...")
n_samples = 1000
sequence_length = 20
n_features = 1

X_train, y_train = generate_sine_sequence(n_samples, sequence_length, n_features)
X_test, y_test = generate_sine_sequence(200, sequence_length, n_features)

print(f"Training data shape: X={X_train.shape}, y={y_train.shape}")
print(f"Test data shape: X={X_test.shape}, y={y_test.shape}")

# Create and train the model
model = create_simple_rnn_model(
    input_shape=(sequence_length-1, n_features),
    hidden_units=32,
    output_units=1
)

print("\nModel Architecture:")
model.summary()

# Train the model
print("\nTraining the RNN model...")
history = model.fit(
    X_train, y_train,
    batch_size=32,
    epochs=20,
    validation_data=(X_test, y_test),
    verbose=1
)


In [ ]:
# Evaluate and visualize results
def plot_training_history(history):
    """Plot training and validation loss"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # Plot loss
    ax1.plot(history.history['loss'], label='Training Loss')
    ax1.plot(history.history['val_loss'], label='Validation Loss')
    ax1.set_title('Model Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Plot MAE
    ax2.plot(history.history['mae'], label='Training MAE')
    ax2.plot(history.history['val_mae'], label='Validation MAE')
    ax2.set_title('Model MAE')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('MAE')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

def visualize_predictions(model, X_test, y_test, n_examples=5):
    """Visualize model predictions on test sequences"""
    predictions = model.predict(X_test[:n_examples])
    
    fig, axes = plt.subplots(n_examples, 1, figsize=(12, 2*n_examples))
    if n_examples == 1:
        axes = [axes]
    
    for i in range(n_examples):
        sequence = X_test[i].flatten()
        true_next = y_test[i]
        pred_next = predictions[i][0]
        
        # Plot the sequence
        axes[i].plot(range(len(sequence)), sequence, 'b-o', label='Input Sequence')
        axes[i].plot(len(sequence), true_next, 'ro', markersize=10, label=f'True Next: {true_next:.3f}')
        axes[i].plot(len(sequence), pred_next, 'go', markersize=10, label=f'Predicted: {pred_next:.3f}')
        
        axes[i].set_title(f'Sequence {i+1} - Error: {abs(true_next - pred_next):.3f}')
        axes[i].legend()
        axes[i].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Plot training history
plot_training_history(history)

# Evaluate model
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=0)
print(f"\nTest Loss: {test_loss:.4f}")
print(f"Test MAE: {test_mae:.4f}")

# Visualize predictions
visualize_predictions(model, X_test, y_test, n_examples=3)
